<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/Untitled50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neqsim==2.5.35

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 10.4 MB/s eta 0:00:00


In [ ]:
from neqsim.thermo import fluid, phaseenvelope
from neqsim.process import separator, heater, clearProcess, stream,  compressor, runProcess, viewProcess, getProcess, splitter, mixer, recycle2, valve, virtualstream, cooler
from jpype.types import JDouble

temperature_inlet = 35.0
pressure_inlet = 55.0
pressure_outlet = 100.0
gasFlowRate = 5.0 


fluid1 = fluid('srk')
fluid1.addComponent("methane", 1.0)

clearProcess()

stream1 = stream(fluid1)
stream1.setPressure(pressure_inlet, 'bara')
stream1.setTemperature(temperature_inlet, 'C')
stream1.setFlowRate(gasFlowRate, "MSm3/day")

streamresycl  = stream(stream1.clone())
streamresycl.setFlowRate(1.0, 'MSm3/day')

mixerStream = mixer()
mixerStream.addStream(stream1)
mixerStream.addStream(streamresycl)

cooler1 = cooler(mixerStream.getOutletStream())
cooler1.setOutTemperature(temperature_inlet, 'C')

compressor_1 = compressor(cooler1.getOutletStream(), pressure_outlet)
compressor_1.setIsentropicEfficiency(0.77)

stream2 = stream(compressor_1.getOutStream())

streamSplit = splitter(stream2)
streamSplit.setFlowRates(JDouble[:]([5, 0.1]), 'MSm3/day')

resycStream1 = streamSplit.getSplitStream(1)

valve1 = valve(resycStream1)
valve1.setOutletPressure(pressure_inlet)
valve1.setPercentValveOpening(5.0)

resycleOp = recycle2()
resycleOp.addStream(valve1.getOutletStream())
resycleOp.setOutletStream(streamresycl)
resycleOp.setFlowAccuracy(1e-6)

exportStream = stream(streamSplit.getSplitStream(0))

virtualStream1 = virtualstream(exportStream)
virtualStream1.setFlowRate(10.0, 'MSm3/day')

runProcess()
print('export flow ' , exportStream.getFlowRate('MSm3/day'))
print('recycle flow ' , resycStream1.getFlowRate('MSm3/day'))
print('flow to compressor ' , mixerStream.getOutStream().getFlowRate('MSm3/day'))
print('power ', compressor_1.getPower('kW'), ' kW')
print('valve Cv ', valve1.getCv())
print('valve opening', valve1.getPercentValveOpening(), ' %')

print('virtual stream flow ' , virtualStream1.getOutStream().getFlowRate('MSm3/day'))



export flow  5.000000002557359
recycle flow  0.1000000000511473
flow to compressor  5.100000002608504
power  5009.597839665677  kW
valve Cv  954.2922257865447
valve opening 4.250000002173752  %
virtual stream flow  10.0


In [ ]:
streamSplit.setFlowRates(JDouble[:]([-1, 1.5]), 'MSm3/day')
runProcess()

print('export flow ' , exportStream.getFlowRate('MSm3/day'))
print('recycle flow ' , resycStream1.getFlowRate('MSm3/day'))
print('flow to compressor ' , mixerStream.getOutStream().getFlowRate('MSm3/day'))
print('power ', compressor_1.getPower('kW'), ' kW')
print('valve Cv ', valve1.getCv())
print('valve opening ', valve1.getPercentValveOpening())

export flow  5.0
recycle flow  1.5
flow to compressor  6.5
power  6384.781557092568  kW
valve Cv  954.2922257865447
valve opening  63.74999999999989


# Test compressor med recycle
FLow recycle regnes som Cv*dP.

In [4]:
from neqsim.thermo import fluid, phaseenvelope
from neqsim.process import separator, heater, clearProcess, stream,  compressor, runProcess, viewProcess, getProcess, splitter, mixer, recycle2, valve, virtualstream, cooler
from jpype.types import JDouble

temperature_inlet = 35.0
pressure_inlet = 55.0
pressure_outlet = 100.0
gasFlowRate = 5.0 
Cv = 0.05

fluid1 = fluid('srk')
fluid1.addComponent("methane", 1.0)

clearProcess()

stream1 = stream(fluid1)
stream1.setPressure(pressure_inlet, 'bara')
stream1.setTemperature(temperature_inlet, 'C')
stream1.setFlowRate(gasFlowRate, "MSm3/day")

flowResycl = Cv * (pressure_outlet-pressure_inlet)

streamresycl  = stream(stream1.clone())
streamresycl.setFlowRate(flowResycl, 'MSm3/day')

mixerStream = mixer()
mixerStream.addStream(stream1)
mixerStream.addStream(streamresycl)

cooler1 = cooler(mixerStream.getOutletStream())
cooler1.setOutTemperature(temperature_inlet, 'C')

compressor_1 = compressor(cooler1.getOutletStream(), pressure_outlet)
compressor_1.setIsentropicEfficiency(0.77)

stream2 = stream(compressor_1.getOutStream())

streamSplit = splitter(stream2)
streamSplit.setFlowRates(JDouble[:]([gasFlowRate, flowResycl]), 'MSm3/day')

exportStream = stream(streamSplit.getSplitStream(0))

runProcess()
print('export flow ' , exportStream.getFlowRate('MSm3/day'))
print('recycle flow ' , streamresycl.getFlowRate('MSm3/day'))
print('flow to compressor ' , mixerStream.getOutStream().getFlowRate('MSm3/day'))

export flow  5.000000000000001
recycle flow  2.25
flow to compressor  7.25
